## PART 3: 3D Scanning

In [1]:
## Initialize everything

%run common.ipynb

Importing libraries...
Done!
Initialize camera and default settings...
Done!
Define functions needed to capture images...
Done!
Generating patterns for projector...
Done!


In [2]:
# We can change the brightness of our projector here.
# Try setting this value to some integer between 0 and 255.
projector_brightness = 255;

# Create a projection pattern
@widgets.interact_manual(brightness=widgets.IntSlider(min=0, max=255, step=1, value=255), 
                         pattern_id=widgets.Dropdown(
    options=[('All Black', 0), ('All White', 1), ('Horiz. Gradient', 2), ('Vert. Gradient', 3), ('Checkerboard', 4)],
    value=4,
    description='Pattern type',
))
def f(brightness, pattern_id):
    global projector_brightness;
    lightcrafter.set_pattern(brightness * patterns[:,:,pattern_id]);
    projector_brightness = brightness;
    time.sleep(1);

    image = numpy.double(get_color_image()) / 255.0;
    plt.imshow(image);
    plt.show();
    return;

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEludFNsaWRlcih2YWx1ZT0yNTUsIGRlc2NyaXB0aW9uPXUnYnJpZ2h0bmVzcycsIG1heD0yNTUpLCBEcm9wZG93bihkZXNjcmlwdGlvbj11J1BhdHRlcm7igKY=


In [3]:
def scan_3D():
    # Create a projection pattern
    pattern_x, pattern_y = numpy.meshgrid(numpy.arange(640),numpy.arange(360))
    pattern_x = numpy.uint16(pattern_x)
    pattern_y = numpy.uint16(pattern_y)

    # Convert to gray codes
    pattern_x = numpy.bitwise_xor(pattern_x,numpy.right_shift(pattern_x,1))

    img = numpy.uint16(numpy.zeros((200,320)))

    for itr in numpy.arange(10):
        ptn1 = numpy.right_shift(numpy.bitwise_and(pattern_x,2**itr),itr);
        ptn2 = 1-ptn1;

        lightcrafter.set_pattern(projector_brightness * ptn1)
        time.sleep(1)
        img1 = numpy.uint16(256*get_raw_image())

        lightcrafter.set_pattern(projector_brightness * ptn2)
        time.sleep(1)
        img2 = numpy.uint16(256*get_raw_image())

        tmp = numpy.uint16(numpy.greater(img1,img2));
        tmp = numpy.left_shift(tmp,itr);
        img = numpy.bitwise_or(img, tmp)

    # Convert to binary codes
    mask = numpy.right_shift(img,1)
    for itr in numpy.arange(10):
        img = numpy.bitwise_xor(img, mask)
        mask = numpy.right_shift(mask,1)

    lightcrafter.set_pattern(projector_brightness * numpy.ones((360,640)));
    time.sleep(1);
    mask = get_raw_image();
        
    return img, mask;

In [4]:
## Capture background plane

correspondences1 = 0;
mask1 = 0;

class background_corresondences(widgets.Button):
    output = widgets.Output()
    
    @output.capture()
    def on_button_clicked(b):
        clear_output();
        
        global correspondences1, mask1;
        correspondences1, mask1 = scan_3D();

        plt.imshow(correspondences1);
        plt.show();
        
button1 = background_corresondences(description="Capture background")
button1.on_click(background_corresondences.on_button_clicked)

display(button1)
display(background_corresondences.output)

background_corresondences(description=u'Capture background', style=ButtonStyle())

Output()

In [5]:
## Capture foreground plane

correspondences2 = 0;
mask2 = 0;

class foreground_corresondences(widgets.Button):
    output = widgets.Output()
    
    @output.capture()
    def on_button_clicked(b):
        clear_output();
        
        global correspondences2, mask2;
        correspondences2, mask2 = scan_3D();

        plt.imshow(correspondences2);
        plt.show();
        
button2 = foreground_corresondences(description="Capture foreground")
button2.on_click(foreground_corresondences.on_button_clicked)

display(button2)
display(foreground_corresondences.output)

foreground_corresondences(description=u'Capture foreground', style=ButtonStyle())

Output()

In [6]:
@widgets.interact_manual(threshold=widgets.FloatSlider(min=0, max=100, step=1, value=0), 
                         avg_val=widgets.FloatSlider(min=0, max=100, step=1, value=50), 
                         range_val=widgets.FloatSlider(min=0, max=100, step=1, value=50))
def g(threshold, avg_val, range_val):
    mask = numpy.double(numpy.greater(numpy.minimum(mask1,mask2),threshold));

    depth = (numpy.double(correspondences2) - numpy.double(correspondences1))
    depth = depth*mask;

    plt.imshow(depth[0::2,0::2],vmin=avg_val-range_val,vmax=avg_val+range_val)
    plt.show()

aW50ZXJhY3RpdmUoY2hpbGRyZW49KEZsb2F0U2xpZGVyKHZhbHVlPTAuMCwgZGVzY3JpcHRpb249dSd0aHJlc2hvbGQnLCBzdGVwPTEuMCksIEZsb2F0U2xpZGVyKHZhbHVlPTUwLjAsIGRlc2PigKY=
